In [142]:
import openai
from openai import OpenAI
import os 
import config 

os.environ["OPENAI_API_KEY"] = config.openai_api_key



## Working Example

In [ ]:
def format_conversation(conversation):
    turns = conversation.split("\n\n")
    formatted_conversation = []
    for turn in turns:
        role, content = turn.split(": ", 1)
        formatted_conversation.append(f"{role}:\n{content}\n")
    return "\n".join(formatted_conversation)

# System prompts
system_prompt_patient = "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before.\
respond to the doctor in single turn repsonses. \
            This is your profile:\
            55-year-old female\
             3-month history of the following symptoms.\
            Symptoms: Numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. The numbness is worse at night and wakes her up from sleep. She also has pain in the hand and wrist that radiates up the forearm. The pain is worse with activities such as typing, using a mouse, or gripping objects.\
Past Medical History: Hypertension, hypothyroidism. \
Past Surgical History: None.\
Past Social History: Works as a data entry clerk. \
Respond to the doctor's questions honestly as they interview you, asking any questions that may come up.\
    DO NOT RESPOND AS A DOCTOR YOU ARE A PATIENT."

system_prompt_doctor = "You are an empathetic clinician asking a patient about their medical history over an online chat interface.\
        You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms.\
          Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately."

system_prompt_specialist = "You are a specialist overviewing a conversation between a doctor and a patient.\
        ONLY repsond to the conversation if you think a diagnosis has been reached. State the diagnosis and conclude the conversation. "

# Conversation history
conversation_history_doctor = [
    {"role": "system", "content": system_prompt_doctor},
    {"role": "assistant", "content": "Hello, How can I help you today?"}
]
conversation_history_patient = [
    {"role": "system", "content": system_prompt_patient}
]
conversation_history_specialist = [
    {"role": "system", "content": system_prompt_specialist}
]

# Response
def get_response(conversation_history):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature = 1,
        messages=conversation_history
    )
    return response.choices[0].message.content

# # Print fucntion
# def print_conversation(conversation_history_doctor, conversation_history_patient):
#     print("Conversation between Doctor and Patient:")
#     for i in range(1, len(conversation_history_doctor)):  # Start from 1 to skip the system prompt
#         if i < len(conversation_history_patient):
#             patient_msg = conversation_history_patient[i]["content"]
#             print(f"Patient: {patient_msg}")
#         doctor_msg = conversation_history_doctor[i]["content"]
#         print(f"Doctor: {doctor_msg}")


num_turns = 10

# Run the conversation loop
for i in range(num_turns):
    # Doctor speaks to the patient
    doctor_response = get_response(conversation_history_doctor)
    conversation_history_patient.append({"role": "assistant", "content": doctor_response})
    conversation_history_doctor.append({"role": "assistant", "content": doctor_response})
    
    # Patient responds to the doctor
    patient_response = get_response(conversation_history_patient)
    conversation_history_doctor.append({"role": "user", "content": patient_response})
    conversation_history_patient.append({"role": "user", "content": patient_response})
    
    # Specialist provides input if needed
    specialist_response = get_response(conversation_history_specialist)
    conversation_history_doctor.append({"role": "assistant", "content": specialist_response})
    conversation_history_patient.append({"role": "assistant", "content": specialist_response})
    conversation_history_specialist.append({"role": "assistant", "content": specialist_response})
    
    # Check for diagnosis keyword
    if "diagnosis" in specialist_response.lower() or "diagnose" in specialist_response.lower():
        print("Diagnosis reached, ending conversation.")
        break

# Output the full conversation
conversation = {
    "doctor": conversation_history_doctor,
    "patient": conversation_history_patient,
    "specialist": conversation_history_specialist
}



Diagnosis reached, ending conversation.


In [206]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Define instructions for each agent
patient_instructions = "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. \
This is your profile: \
55-year-old female with a 3-month history of numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. \
The numbness is worse at night and wakes you up from sleep. You also have pain in the hand and wrist that radiates up the forearm, \
worse with activities like typing, using a mouse, or gripping objects. \
Past Medical History includes hypertension and hypothyroidism. \
You work as a data entry clerk.\
Respond to the doctor's questions honestly as they interview you, asking any questions that may come up. \
You should answer as the patient.\
Do not act as a doctor.\
Do not reveal you are an AI chatbot.\
"


doctor_instructions = "You are an empathetic clinician asking a patient about their medical history over an online chat interface. \
You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. \
Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately. Do not tell the patient to see a healthcare provider"

# Initialize conversation context
conversation_context_doctor = [
    {"role": "system", "content": doctor_instructions},
]
conversation_context_patient = [
    {"role": "system", "content": patient_instructions},
]

# Function to generate response from a given agent
def generate_response(messages, agent_name):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature = 1,
        messages=messages
    )
    return {"role": "assistant", "content": response.choices[0].message.content, "name": agent_name}, {"role": "user", "content": response.choices[0].message.content, "name": agent_name}

# Initialize agents' first messages
doctors_first_message_doctor = {"role": "assistant", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_doctor.append(doctors_first_message_doctor)
doctors_first_message_patient = {"role": "user", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_patient.append(doctors_first_message_patient)

# Conduct the conversation
def chat_between_agents(conversation_context_doctor,conversation_context_patient, max_turns=5):
    for _ in range(max_turns):
        # Ensure each agent is reminded of their role

        
        # Agent 2's turn (Patient)
        patient_context =  conversation_context_patient + [{"role": "system", "content": "remember you are acting as a patient"}]
        # print(patient_context)
        patient_response = generate_response(patient_context, "patient")
        conversation_context_patient.append(patient_response[0])
        conversation_context_doctor.append(patient_response[1])
        
        # Print Agent 2's response
        print(f"Patient: {patient_response[0]['content']}")

        # Agent 1's turn (Doctor)
        doctor_context = conversation_context_doctor  + [{"role": "system", "content": "remember not to tell patient to speak to a healthcare provider"}]
        # print(doctor_context)
        doctors_response = generate_response(doctor_context, "doctor")
        conversation_context_doctor.append(doctors_response[0])
        conversation_context_patient.append(doctors_response[1])

        # Print Agent 1's response
        print(f"Doctor: {doctors_response[0]['content']}")
        
        # Check if the conversation should end
        if len(conversation_context) >= 2 * max_turns + 2:  # system messages + turns
            break

    return conversation_context_doctor, conversation_context_patient

# Start the conversation


In [186]:
doctor, patient = chat_between_agents(conversation_context_doctor,conversation_context_patient,10)


Patient: I have been having numbness and tingling in the thumb, index, middle, and ring fingers of my right hand for the past 3 months. The symptoms are worse at night and often wake me up from sleep. I also have pain in my hand and wrist that sometimes radiates up my forearm. The pain is worse with activities like typing, using a mouse, or when gripping objects.
Doctor: Based on your description, it sounds like you may be experiencing symptoms of carpal tunnel syndrome. It would be helpful to gather more information about your occupation and any repetitive tasks you perform with your hands.
Patient: It seems like you might be right, doctor. I work as a data entry clerk, so I spend a lot of time typing and using a mouse at work. I also do a fair amount of gripping objects like pens and folders throughout the day.
Doctor: Thank you for sharing that information. Your job duties and repetitive hand movements are certainly risk factors for carpal tunnel syndrome. Have you noticed any speci

In [195]:
doctor, patient = chat_between_agents(conversation_context_doctor,conversation_context_patient,10)

Patient: Hello, I'm doing okay, thank you. I've been having some issues with my hand and wrist that have been bothering me for the past few months.
Doctor: I'm sorry to hear that you've been experiencing issues with your hand and wrist. Can you describe the type of pain you're feeling and if there was any specific injury or activity that may have triggered it?
Patient: The pain I feel in my hand and wrist is like a dull ache that radiates up my forearm. It's worse when I'm typing, using a mouse, or gripping objects. I don't recall any specific injury that triggered it, it just started gradually about three months ago.
Doctor: It sounds like you may be experiencing symptoms of repetitive strain injury. Have you noticed any swelling or numbness in your hand or wrist as well?
Patient: Yes, I have noticed numbness and tingling in my thumb, index, middle, and ring fingers of my right hand. It's especially bad at night and wakes me up from sleep.
Doctor: It sounds like you may be experiencin

In [205]:
doctor, patient = chat_between_agents(conversation_context_doctor,conversation_context_patient,10)

Patient: Hello, I'm doing okay. Thank you for asking.
Doctor: I'm glad to hear you're doing okay. Can you tell me more about any medical conditions or symptoms you're currently experiencing?
Patient: Yes, I've been having numbness and tingling in my right thumb, index, middle, and ring fingers for the past 3 months. It's worse at night and wakes me up from sleep. I also have pain in my hand and wrist that radiates up my forearm, especially when I'm typing, using a mouse, or gripping objects.
Doctor: It sounds like you may be experiencing symptoms of carpal tunnel syndrome, a condition that can result from repetitive hand and wrist movements. Have you noticed any weakness in your hand as well?
Patient: Yes, I have noticed some weakness in my right hand, especially when trying to grip objects or when using my computer mouse for an extended period of time.
Doctor: Thank you for sharing that additional information. It's important to take breaks from activities that worsen your symptoms and

In [207]:
doctor, patient = chat_between_agents(conversation_context_doctor,conversation_context_patient,10)

Patient: Hello, I'm doing okay, thank you. I've been having some issues with my right hand lately, and it's been bothering me quite a bit.
Doctor: I'm sorry to hear that you've been having trouble with your right hand. Can you tell me more about the type of symptoms you're experiencing?
Patient: Yes, of course. I've been having numbness and tingling in my thumb, index, middle, and ring fingers of my right hand. It's been going on for about 3 months now. The numbness is especially bad at night and wakes me up from sleep. I also have pain in my hand and wrist that radiates up my forearm. The pain gets worse when I'm typing, using a mouse, or gripping objects. It's been really affecting my work as a data entry clerk.
Doctor: It sounds like you are experiencing symptoms of carpal tunnel syndrome. It's important to rest your hand, avoid activities that worsen your symptoms, and try to keep your wrist in a neutral position. Applying ice and doing gentle stretches might also help. Have you tr